In [1]:
import json
import os
from os.path import join as pjoin
import cv2

In [2]:
def draw_label(img, bound, color, text=None, put_text=True):
    cv2.rectangle(img, (bound[0], bound[1]), (bound[2], bound[3]), color, 3)
    if put_text:
        # put text with rectangle 
        (w,h),_ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.6, 3)
        cv2.rectangle(img, (bound[0], bound[1] - 40), (bound[0] + w, bound[1] - 40 + h), color, -1)
        cv2.putText(img, text, (bound[0], bound[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3) 


def visualize_list(img, data, show=True):
    board = img.copy()
    is_drawn = False
    
    if data['vertical_groups'] is not None:
        for g in data['vertical_groups']:
            draw_label(board, data['bounds'][g[0]], (0,0,166), 'vertical_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (0,0,166), put_text=False)
                is_drawn = True

    if data['horizontal_groups'] is not None:
        for g in data['horizontal_groups']:
            draw_label(board, data['bounds'][g[0]], (166,0,0), 'horizontal_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,0,0), put_text=False)
                is_drawn = True
                
    if data['multitab_groups'] is not None:
        for g in data['multitab_groups']:
            draw_label(board, data['bounds'][g[0]], (166,0,166), 'multitab_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,0,166), put_text=False)
                is_drawn = True
    
    if data['list_groups'] is not None:
        for g in data['list_groups']:
            draw_label(board, data['bounds'][g[0]], (166,100,255), 'list_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,100,255), put_text=False)
                is_drawn = True        
                
    if data['pageindcator_groups'] is not None:
        for g in data['pageindcator_groups']:
            draw_label(board, data['bounds'][g[0]], (166,166,166), 'pageindcator_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,166,166), put_text=False)
                is_drawn = True     
        
    if show and is_drawn:
        cv2.imshow('data', cv2.resize(board, (500, 800)))
        key = cv2.waitKey(0)
        return key
    return None


def visualize_results(image_name):
    uied_result = cv2.imread(pjoin(result_root, 'uied', image_name + '.jpg'))
    layout_result = cv2.imread(pjoin(result_root, 'layout', image_name + '-list.jpg'))
    cv2.imshow('uied', uied_result)
    cv2.imshow('layout', layout_result)
    

def visualize_ground_truth(image_name, gt_data):
    img_file = pjoin(data_root, image_name + '.jpg')
    img_data = gt_data[image_name]
    img = cv2.imread(img_file)
    
    org_json_file = pjoin(data_root, image_name + '.json')
    org_data = json.load(open(org_json_file))
    org_shape = org_data['activity']['root']['bounds']
    
    img = cv2.resize(img, (1440, 2560))
    return visualize_list(img, img_data)

In [3]:
def cvt_list_gt(gt_data):
    lst = sorted(gt_data['list'].items(), key=lambda x: x[0])
    lst = [l[1] for l in lst]
    return lst

In [5]:
def evaluate(gt, result):
    print('*** Ground Truth ***')
    print('Number of list items:', len(gt))
    for item in gt:
        print(item)
    print('*** Result ***')
    print('Number of lists:', len(result))
    for lst in result:
        print('Number of list items:', len(lst))
        for item in lst:
            print(item)

In [6]:
data_root = 'E:\\Mulong\\Datasets\\gui\\rico\\combined\\all'
result_root = 'E:\\Temp\\data\\rico-layout\\result'
list_gt = json.load(open('item_data_class.json'))
gui_gt = json.load(open('valid_group_data.json'))

In [8]:
start_point = None
start = False
for img_name in open('select.txt', 'r').readlines()[:200]:
    img_name = img_name.replace('\n', '')
    if start_point is None or img_name == start_point: start = True
    if not start: continue
    print('\n', img_name)
    
    item_gt = cvt_list_gt(list_gt[img_name])
    item_result = json.load(open(pjoin('E:\\Temp\\data\\rico-layout\\result', 'layout', img_name + '-list.json')))['list']
    evaluate(item_gt, item_result)
    
    visualize_results(img_name)
    key = visualize_ground_truth(img_name, gui_gt)
    if key is not None and key != -1 and chr(key) == 'q':
        break
cv2.destroyAllWindows()


 2114
*** Ground Truth ***
Number of list items: 10
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView', 'TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 10
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Text']
['Text']


In [29]:
img_name = '26644'

item_gt = cvt_list_gt(list_gt[img_name])
item_result = json.load(open(pjoin('E:\\Temp\\data\\rico-layout\\result', 'layout', img_name + '-list.json')))['list']
evaluate(item_gt, item_result)

visualize_results(img_name)
visualize_ground_truth(img_name, gui_gt)
cv2.destroyAllWindows()

*** Ground Truth ***
Number of list items: 9
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
['ImageView', 'TextView', 'ImageView', 'ImageView']
*** Result ***
Number of lists: 1
Number of list items: 9
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Compo', 'Text']
['Compo', 'Compo', 'Text']
